<a href="https://colab.research.google.com/github/MichaelRuns/MichaelRuns/blob/main/AOS_to_XYZ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tf2onnx onnx
!mkdir models

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 451.2/451.2 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 53.2 MB/s eta 0:00:00
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 23.3.3
    Uninstalling flatbuffers-23.3.3:
      Successfully uninstalled flatbuffers-23.3.3


In [2]:
from google.colab import drive
import tensorflow as tf
import cv2
import numpy as np
import re
from sklearn.model_selection import train_test_split
import tf2onnx
import onnx
import argparse
import os.path
import glob
import re
from typing import *
import shutil

file I/O helper func

In [3]:
class Entry:
    def __init__(self, blade_index: int, offset_xyz: Tuple[int,int,int], himag_cam_name: Literal['a', 'b', 'c', 'd']) -> None:
        self.blade_index = blade_index
        self.offset_xyz = offset_xyz
        self.himag_cam_name = himag_cam_name

    @property
    def is_left(self):
        return self.himag_cam_name in ('a', 'b')

    @property
    def is_right(self):
        return not self.is_left

    @property
    def is_far(self):
        return self.himag_cam_name in ('a', 'd')

    @property
    def is_near(self):
        return not self.is_far

    def __hash__(self):
        return hash((
            self.blade_index,
            self.offset_xyz,
            self.himag_cam_name
        ))
    
    def __str__(self) -> str:
        return f"(b={self.blade_index},o={self.offset_xyz},cam={repr(self.himag_cam_name)})"

    def __repr__(self) -> str:
        return str(self)


def gen_param_to_image_map(image_root):
    glob_pattern = "Reading-B*-2023-03-17-2022-(*)_HighMag*.png"
    match_file_path_list = glob.glob(f"{image_root}/{glob_pattern}")

    output = {}
    re_pattern = re.compile(r"^Reading-B(\d+)-2023-03-17-2022-\((-?\d+),(-?\d+),(-?\d+)\)_HighMag([ABCD])\.png$")
    for match_file_path in match_file_path_list:
        match_file_name = os.path.basename(match_file_path)
        match_obj = re_pattern.match(match_file_name)
        assert match_obj
        blade_index = int(match_obj.group(1))
        offset_x = int(match_obj.group(2))
        offset_y = int(match_obj.group(3))
        offset_z = int(match_obj.group(4))
        himag_cam_name = match_obj.group(5).lower()

        entry = Entry(
            blade_index=blade_index,
            offset_xyz=(offset_x, offset_y, offset_z),
            himag_cam_name=himag_cam_name
        )
        #output[entry] = match_file_path
        output[match_file_path] = entry
    return output

copy data to runtime

In [4]:
drive.mount('/content/drive')
source_file_path = '/content/drive/MyDrive/AOS_DATA'
dest_file_path = '/content/data'
shutil.copytree(source_file_path, dest_file_path)

Mounted at /content/drive


'/content/data'

define hyperparams

In [5]:
IMG_WIDTH = 256
IMG_HEIGHT = 192
DATA_PATHS = ['data/Dataset-003/HighMagCalibration', 'data/Dataset-004/HighMagCalibration','data/Dataset-005/HighMagCalibration', 'data/Dataset-006/HighMagCalibration', 'data/Dataset-009/HighMagCalibration', 'data/Dataset-010/HighMagCalibration', 'data/Dataset-011/HighMagCalibration']
OUTPUT_DIMS = 3
BATCH_SIZE = 32
EPOCHS = 25
MODEL_NAME = 'multi-modal-2'

In [6]:
def make_model():
    input1 = tf.keras.Input(shape=(IMG_WIDTH, IMG_HEIGHT, 1), name='input1')
    input2 = tf.keras.Input(shape=(IMG_WIDTH, IMG_HEIGHT, 1), name='input2')
    cnn1 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu')(input1)
    cnn1 = tf.keras.layers.MaxPooling2D((2, 2))(cnn1)
    cnn1 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu')(cnn1)
    cnn1 = tf.keras.layers.MaxPooling2D((2, 2))(cnn1)
    cnn1 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu')(cnn1)
    cnn1 = tf.keras.layers.Flatten()(cnn1)
    cnn2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu')(input2)
    cnn2 = tf.keras.layers.MaxPooling2D((2, 2))(cnn2)
    cnn2 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu')(cnn2)
    cnn2 = tf.keras.layers.MaxPooling2D((2, 2))(cnn2)
    cnn2 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu')(cnn2)
    cnn2 = tf.keras.layers.Flatten()(cnn2)
    combined = tf.keras.layers.concatenate([cnn1, cnn2])
    dense = tf.keras.layers.Dense(128, activation='relu')(combined)
    dense = tf.keras.layers.Dropout(0.5)(dense)
    dense = tf.keras.layers.Dense(64, activation='relu')(dense)
    dense = tf.keras.layers.Dense(32, activation='relu')(dense)
    dense = tf.keras.layers.Dropout(0.5)(dense)
    dense = tf.keras.layers.Dense(16, activation='relu')(dense)
    output = tf.keras.layers.Dense(OUTPUT_DIMS, activation='linear')(dense)
    model = tf.keras.Model(inputs=[input1, input2], outputs=output)
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [7]:
def readData():
    dataHash = set()
    inputs = []
    labels = []
    for path in DATA_PATHS:
        path = '/content/' + path
        mp = gen_param_to_image_map(path)
        for key in mp:
            dataHash.add(key)
        print(f'{path} finished')
    print(f'{len(dataHash)} images loaded into hash')
    print(f'there are {len(dataHash)/4} quads')
    # quad_keys = [list(dataHash.keys())[i:i+4] for i in range(0, len(dataHash), 4)]
    delims = "[(|)]"
    seen = set()
    for key in dataHash:
      pairID = key[:-13]
      print(f'pairID: {pairID}')
      seen.add(pairID)
      chunk = {}
      for cam in ['A','B','C','D']:
        camKey = pairID + '_HighMag' + cam + '.png'
        label = re.split(delims, camKey)[1].split(',')
        label = [int(i) for i in label]
        img = cv2.imread(camKey, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (IMG_HEIGHT, IMG_WIDTH))
        img = img.astype(np.float32) / 255.0
        chunk[cam] = tuple(label), img
      inputs.append((chunk['A'][1], chunk['B'][1]))
      labels.append(chunk['A'][0])
      inputs.append((chunk['D'][1], chunk['C'][1]))
      labels.append(chunk['D'][0])
    inputs = np.array(inputs)
    labels = np.array(labels)
    print(f'inputs shape: {inputs.shape}')
    print(f'labels shape: {labels.shape}')
    X_train, X_test, y_train, y_test = train_test_split(inputs, labels, test_size=0.2)
    print('data split')
    print(f'X_train shape: {X_train.shape}')
    print(f'X_test shape: {X_test.shape}')
    print(f'y_train shape: {y_train.shape}')
    print(f'y_test shape: {y_test.shape}')
    print('training model now')
    return X_train, X_test, y_train, y_test

In [8]:
def main():
    model = make_model()
    earlystop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto')
    x_train, x_test, y_train, y_test = readData()
    model.fit([x_train[:,0], x_train[:,1]], y_train, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=([x_test[:,0], x_test[:,1]], y_test), callbacks=[earlystop])
    print('model trained')
    model.save(f'/content/drive/MyDrive/AOS_DATA/models/{MODEL_NAME}.h5')
    onnx_model, _ = tf2onnx.convert.from_keras(model)
    onnx.save(onnx_model, f'/content/drive/MyDrive/AOS_DATA/models/{MODEL_NAME}.onnx')
    print('model saved')

In [ ]:
main()

/content/data/Dataset-003/HighMagCalibration finished
/content/data/Dataset-004/HighMagCalibration finished
/content/data/Dataset-005/HighMagCalibration finished
/content/data/Dataset-006/HighMagCalibration finished
/content/data/Dataset-009/HighMagCalibration finished
/content/data/Dataset-010/HighMagCalibration finished
/content/data/Dataset-011/HighMagCalibration finished
1500 images loaded into hash
there are 375.0 quads
pairID: /content/data/Dataset-009/HighMagCalibration/Reading-B5-2023-03-17-2022-(0,50,0)
pairID: /content/data/Dataset-010/HighMagCalibration/Reading-B5-2023-03-17-2022-(-100,-50,0)
pairID: /content/data/Dataset-009/HighMagCalibration/Reading-B5-2023-03-17-2022-(-50,-100,100)
pairID: /content/data/Dataset-011/HighMagCalibration/Reading-B3-2023-03-17-2022-(0,0,0)
pairID: /content/data/Dataset-009/HighMagCalibration/Reading-B5-2023-03-17-2022-(100,-100,100)
pairID: /content/data/Dataset-009/HighMagCalibration/Reading-B5-2023-03-17-2022-(-100,0,50)
pairID: /content/da